In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### Dataset preparation:

- Read train dataset
- Create validation dataset
- Create Pytorch custom dataset preparation class
- Create Pytorch dataloaders



In [2]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train.shape

(42000, 785)

In [5]:
train_df , val_df = train_test_split(train,test_size=0.2 , random_state=0,shuffle=True)

In [6]:
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [7]:
class CustomDataset(Dataset):
    def __init__(self , data , dataset_type):
        self.images = data.iloc[:,1:].values
        self.labels = data.iloc[: , 0].values
        self.dataset_type = dataset_type
        self.train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])])
        self.valid_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self,idx):
        image = self.images[idx].reshape(28,28).astype(np.uint8)
        label = self.labels[idx]
        if self.dataset_type=='train':
            image = self.train_transform(image)
        else:
            image = self.valid_transform(image)
        
        return image,label
            

In [8]:
train_ds = CustomDataset(train_df, 'train')
valid_ds = CustomDataset(val_df, 'valid')

In [9]:
len(train_ds) , len(valid_ds)

(33600, 8400)

In [10]:
np.unique(train_ds.labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
BATCH_SIZE = 512
train_dataloader = DataLoader(dataset=train_ds,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

val_dataloader = DataLoader(dataset=valid_ds,
                            batch_size=BATCH_SIZE,
                            shuffle=True)

In [12]:
for image,label in train_dataloader:
    print(image.shape)
    print(label.shape)
    break

torch.Size([512, 1, 28, 28])
torch.Size([512])


### Vision Transformer

In [13]:
import torch
from torch import nn


#### Parameters

In [14]:
NUM_CLASSES = 10
PATCH_SIZE = 4
IMG_SIZE = 28
IN_CHANNELS = 1
NUM_HEADS = 8
DROPOUT = 0.001
HIDDEN_DIM = 768
ACTIVATION="gelu"
NUM_ENCODERS = 8
EMBED_DIM = (PATCH_SIZE ** 2) * IN_CHANNELS # 16
NUM_PATCHES = (IMG_SIZE // PATCH_SIZE) ** 2 # 49

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

#### PatchEmbedding class

In [15]:
class PatchEmbedding(nn.Module):
    def __init__(self, embed_dim, patch_size, num_patches, dropout, in_channels):
        super().__init__()
        self.patcher = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=embed_dim,
                kernel_size=patch_size,
                stride=patch_size,
            ),                  
            nn.Flatten(2))

        self.cls_token = nn.Parameter(torch.randn(size=(1, 1, embed_dim)), requires_grad=True)
        self.position_embeddings = nn.Parameter(torch.randn(size=(1, num_patches+1, embed_dim)), requires_grad=True)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)

        x = self.patcher(x).permute(0, 2, 1)
        x = torch.cat([cls_token, x], dim=1)
        x = self.position_embeddings + x 
        x = self.dropout(x)
        return x
    
model = PatchEmbedding(EMBED_DIM, PATCH_SIZE, NUM_PATCHES, DROPOUT, IN_CHANNELS).to(device)

In [16]:
image.shape

torch.Size([512, 1, 28, 28])

In [17]:
model.eval()
patched = model(image.to(device))

In [18]:
patched.shape

torch.Size([512, 50, 16])

#### Vision Transformer

In [19]:
class VissionTransformer(nn.Module):
    def __init__(self, num_patches, img_size, num_classes, patch_size, embed_dim, num_encoders, 
                 num_heads, hidden_dim, dropout, activation, in_channels):
        super().__init__()
        self.embeddings_block = PatchEmbedding(embed_dim, patch_size, num_patches, dropout, in_channels)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout, 
                                                   activation=activation, batch_first=True, norm_first=True)
        self.encoder_blocks = nn.TransformerEncoder(encoder_layer, num_layers=num_encoders)

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(normalized_shape=embed_dim),
            nn.Linear(in_features=embed_dim, out_features=num_classes)
        )

    def forward(self, x):
        x = self.embeddings_block(x)    # Patch embedding blocks
        x = self.encoder_blocks(x)     # Encoder blocker with Multi-head attention
        x = self.mlp_head(x[:, 0, :])  # Apply classification on the CLS token only - MLP layer
        return x

In [20]:
model = VissionTransformer(NUM_PATCHES, IMG_SIZE, NUM_CLASSES, PATCH_SIZE, EMBED_DIM,
                           NUM_ENCODERS, NUM_HEADS, HIDDEN_DIM,
                           DROPOUT, ACTIVATION, IN_CHANNELS).to(device)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [21]:
model.eval()
mpl_output = model(image.to(device))

In [22]:
mpl_output.shape

torch.Size([512, 10])

In [23]:
mpl_output[0]

tensor([ 0.4043, -0.1069, -0.8049,  0.9896,  0.0604, -1.1779, -0.1905,  0.4683,
         0.5640, -0.6701], device='cuda:0', grad_fn=<SelectBackward0>)

### Loss function and Optimizer

In [24]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Model Training

In [25]:
EPOCHS = 100 ## run the model for 100 epochs

for epoch in tqdm(range(EPOCHS)): 
    train_loss, valid_loss = [], []
    ## training part 
    model.train()
    for images, labels in train_dataloader:
        
        #moving data into the device
        images , labels = images.to(device) , labels.to(device)
        
        optimizer.zero_grad()
        ## 1. forward propagation
        outputs = model(images)
        
        ## 2. loss calculation
        loss = criterion(outputs, labels)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part
    with torch.no_grad():
        model.eval()
        for images, labels in val_dataloader:
            images , labels = images.to(device) , labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss.append(loss.item())
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

  1%|          | 1/100 [00:20<33:47, 20.48s/it]

Epoch: 0 Training Loss:  2.3074351621396616 Valid Loss:  2.2789344647351433


  2%|▏         | 2/100 [00:40<33:14, 20.35s/it]

Epoch: 1 Training Loss:  2.0736881295839944 Valid Loss:  1.8663071534212898


  3%|▎         | 3/100 [01:00<32:36, 20.17s/it]

...

Epoch: 97 Training Loss:  0.040725477801805195 Valid Loss:  0.06836360671064433


 99%|█████████▉| 99/100 [33:05<00:20, 20.09s/it]

Epoch: 98 Training Loss:  0.048382223837754944 Valid Loss:  0.07288752978338915


100%|██████████| 100/100 [33:25<00:00, 20.05s/it]

Epoch: 99 Training Loss:  0.044365706627793385 Valid Loss:  0.0688697697923464


In [26]:
torch.cuda.empty_cache()

### Inference using trained model

In [27]:
predictions = []
model.eval()

with torch.no_grad():
    for images, labels in tqdm(val_dataloader):
        images = images.to(device)
                
        outputs = model(images)
        predictions.extend([int(i) for i in torch.argmax(outputs, dim=1)])

100%|██████████| 17/17 [00:02<00:00,  6.75it/s]


In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(val_df['label'].values,predictions)

0.09785714285714285